In [35]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "duckdb",
#     "polars",
#     "pyarrow",
# ]
#
# [tool.uv]
# exclude-newer = "2025-04-04T15:57:31.848989629+02:00"
# ///

In [36]:
import duckdb
import polars as pl

In [37]:
duckdb.sql("""
CREATE OR REPLACE TABLE anomalous_csv AS
    FROM '../20210103_anomalous_suspicious.csv.parquet';
CREATE OR REPLACE TABLE anomalous_xml AS
    FROM '../20210103_anomalous_suspicious.xml.parquet';
CREATE OR REPLACE TABLE notice_csv AS
    FROM '../20210103_notice.csv.parquet';
CREATE OR REPLACE TABLE notice_xml AS
    FROM '../20210103_notice.xml.parquet';
SHOW TABLES;
""")

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ anomalous     │
│ anomalous_csv │
│ anomalous_xml │
│ notice        │
│ notice_csv    │
│ notice_xml    │
└───────────────┘

In [38]:
duckdb.sql("""
SELECT *
FROM anomalous_csv LIMIT 5;
""")

┌────────────┬────────────────┬──────────┬─────────────────┬──────────┬─────────────┬───────────┬───────────┬───────────┬──────────┐
│ anomaly_id │     src_ip     │ src_port │     dst_ip      │ dst_port │  taxonomy   │ heuristic │ detectors │   label   │ distance │
│   uint16   │    varchar     │  uint16  │     varchar     │  uint16  │   varchar   │  double   │   uint8   │  varchar  │  double  │
├────────────┼────────────────┼──────────┼─────────────────┼──────────┼─────────────┼───────────┼───────────┼───────────┼──────────┤
│          0 │ 77.27.92.100   │     5072 │ 202.65.57.232   │     NULL │ ptpposcaUDP │   0.72041 │        10 │ anomalous │     NULL │
│          1 │ 203.178.149.55 │     NULL │ 206.215.228.128 │     NULL │ ipv4gretun  │   0.35983 │         6 │ anomalous │     NULL │
│          1 │ 203.178.149.55 │     NULL │ 204.167.201.110 │     NULL │ ipv4gretun  │   0.35983 │         6 │ anomalous │     NULL │
│          2 │ NULL           │      443 │ 163.218.81.24   │     NULL

In [39]:
duckdb.sql("""
SELECT *
FROM anomalous_xml LIMIT 5;
""")

┌────────────┬───────────┬────────┬────────┬────────────────┬──────────┬─────────────────┬──────────┬─────────┐
│ anomaly_id │   label   │ start  │  stop  │     src_ip     │ src_port │     dst_ip      │ dst_port │  proto  │
│   uint16   │  varchar  │ uint32 │ uint32 │    varchar     │ varchar  │     varchar     │  uint16  │ varchar │
├────────────┼───────────┼────────┼────────┼────────────────┼──────────┼─────────────────┼──────────┼─────────┤
│          0 │ anomalous │   NULL │   NULL │ 77.27.92.100   │ 5072     │ 202.65.57.232   │     NULL │ NULL    │
│          1 │ anomalous │   NULL │   NULL │ 203.178.149.55 │ NULL     │ 206.215.228.128 │     NULL │ NULL    │
│          1 │ anomalous │   NULL │   NULL │ 203.178.149.55 │ NULL     │ 204.167.201.110 │     NULL │ NULL    │
│          2 │ anomalous │   NULL │   NULL │ NULL           │ 443      │ 163.218.81.24   │     NULL │ NULL    │
│          3 │ anomalous │   NULL │   NULL │ NULL           │ 443      │ 163.218.101.211 │     NULL │ NU

In [40]:
duckdb.sql("""
CREATE OR REPLACE TABLE anomalous AS

WITH xml AS (
  SELECT ROW_NUMBER() OVER () AS rn, * FROM anomalous_xml
),
csv AS (
  SELECT *, ROW_NUMBER() OVER () AS rn FROM anomalous_csv
)

SELECT xml.*, csv.taxonomy, csv.heuristic, csv.detectors, csv.distance
FROM xml
JOIN csv ON xml.rn = csv.rn;
""")

In [41]:
duckdb.sql("""
CREATE OR REPLACE TABLE notice AS

WITH xml AS (
  SELECT ROW_NUMBER() OVER () AS rn, * FROM notice_xml
),
csv AS (
  SELECT *, ROW_NUMBER() OVER () AS rn FROM notice_csv
)

SELECT xml.*, csv.taxonomy, csv.heuristic, csv.detectors, csv.distance
FROM xml
JOIN csv ON xml.rn = csv.rn;
""")

In [42]:
duckdb.sql("FROM ANOMALOUS;")

┌───────┬────────────┬────────────┬────────────┬────────────┬─────────────────┬──────────┬─────────────────┬──────────┬─────────┬───────────────┬───────────┬───────────┬──────────┐
│  rn   │ anomaly_id │   label    │   start    │    stop    │     src_ip      │ src_port │     dst_ip      │ dst_port │  proto  │   taxonomy    │ heuristic │ detectors │ distance │
│ int64 │   uint16   │  varchar   │   uint32   │   uint32   │     varchar     │ varchar  │     varchar     │  uint16  │ varchar │    varchar    │  double   │   uint8   │  double  │
├───────┼────────────┼────────────┼────────────┼────────────┼─────────────────┼──────────┼─────────────────┼──────────┼─────────┼───────────────┼───────────┼───────────┼──────────┤
│     1 │          0 │ anomalous  │       NULL │       NULL │ 77.27.92.100    │ 5072     │ 202.65.57.232   │     NULL │ NULL    │ ptpposcaUDP   │   0.72041 │        10 │     NULL │
│     2 │          1 │ anomalous  │       NULL │       NULL │ 203.178.149.55  │ NULL     │ 206.

In [43]:
duckdb.sql("FROM notice;")

┌───────┬────────────┬─────────┬────────────┬────────────┬─────────────────┬──────────┬────────────────┬──────────┬─────────┬───────────────┬───────────┬───────────┬───────────┐
│  rn   │ anomaly_id │  label  │   start    │    stop    │     src_ip      │ src_port │     dst_ip     │ dst_port │  proto  │   taxonomy    │ heuristic │ detectors │ distance  │
│ int64 │   uint16   │ varchar │   uint32   │   uint32   │     varchar     │ varchar  │    varchar     │  uint16  │ varchar │    varchar    │  double   │   uint8   │  double   │
├───────┼────────────┼─────────┼────────────┼────────────┼─────────────────┼──────────┼────────────────┼──────────┼─────────┼───────────────┼───────────┼───────────┼───────────┤
│     1 │          0 │ notice  │ 1609650300 │ 1609650301 │ 163.218.120.23  │ NULL     │ 99.36.148.14   │      443 │ tcp     │ alphfl        │     504.0 │         1 │ -0.413337 │
│     2 │          1 │ notice  │ 1609650672 │ 1609650679 │ 45.173.170.47   │ 44023    │ NULL           │     N

In [44]:
duckdb.sql("""
SELECT * EXCLUDE rn FROM anomalous
UNION ALL
SELECT * EXCLUDE rn FROM notice;
""")

┌────────────┬───────────┬────────────┬────────────┬─────────────────┬──────────┬─────────────────┬──────────┬─────────┬─────────────┬───────────┬───────────┬───────────┐
│ anomaly_id │   label   │   start    │    stop    │     src_ip      │ src_port │     dst_ip      │ dst_port │  proto  │  taxonomy   │ heuristic │ detectors │ distance  │
│   uint16   │  varchar  │   uint32   │   uint32   │     varchar     │ varchar  │     varchar     │  uint16  │ varchar │   varchar   │  double   │   uint8   │  double   │
├────────────┼───────────┼────────────┼────────────┼─────────────────┼──────────┼─────────────────┼──────────┼─────────┼─────────────┼───────────┼───────────┼───────────┤
│          0 │ anomalous │       NULL │       NULL │ 77.27.92.100    │ 5072     │ 202.65.57.232   │     NULL │ NULL    │ ptpposcaUDP │   0.72041 │        10 │      NULL │
│          1 │ anomalous │       NULL │       NULL │ 203.178.149.55  │ NULL     │ 206.215.228.128 │     NULL │ NULL    │ ipv4gretun  │   0.35983 